In [82]:
import numpy as np

In [83]:
import numpy as np


In [84]:
pip install tick

In [85]:
from tick.hawkes import SimuHawkesExpKernels
from tick.plot import plot_point_process

from tick.plot import plot_hawkes_kernels
from tick.hawkes import SimuHawkesSumExpKernels, SimuHawkesMulti, \
    HawkesSumExpKern

from tick.hawkes import SimuHawkes, HawkesKernelSumExp
n_nodes = 10  # dimension of the Hawkes process
adjacency = 0.2 * np.ones((n_nodes, n_nodes))
adjacency[0, 1] = 0
decays = 3 * np.ones((n_nodes, n_nodes))
baseline = 0.5 * np.ones(n_nodes)
hawkes = SimuHawkes(baseline=baseline, verbose=False, seed=2398)

run_time = 100
hawkes.end_time = run_time
dt = 0.01
hawkes.track_intensity(dt)
hawkes.simulate()
timestamps = hawkes.timestamps

In [86]:
events = timestamps

In [87]:
run_time = 40

hawkes = SimuHawkes(n_nodes=1, end_time=run_time, verbose=False, seed=1398)
kernel = HawkesKernelSumExp([.1, .2, .1], [1., 3., 7.])
hawkes.set_kernel(0, 0, kernel)
hawkes.set_baseline(0, 1.)

dt = 0.01
hawkes.track_intensity(dt)
hawkes.simulate()
timestamps2 = hawkes.timestamps

In [ ]:
events

In [89]:
gamma = 1.04
T_b = 1
T_h = 15* 864
N = 10
grid = []

for i in range(1500):
  a = gamma**i * T_b
  if (a <= T_h):
    grid.append(a)
  else:
    break

grid = np.array(grid)

In [90]:
def fws(p, t1, t2):
  n = sum(list(map(int, (p >= t1) & (p <= t2))))
  return min(int(np.log2(n+1)), 9)

In [92]:
def fws_array(p, array):
  fws_array = ''
  for i in range(1, len(array)):
    fws_array +=str(fws(p, array[i-1], array[i]))
  # fws_array = tuple(fws_array)
  return fws_array

In [93]:
events_clusters = dict()
events_triplets = dict()

In [79]:
n = 20

In [94]:
for i in range(len(grid)):
  for k in range(n):
    Delta_T = np.arange(0, grid[i], 2**k)
    Delta_T = tuple(Delta_T)
    for p_k in events:
      T_j = grid[i]
      triplet = (T_j, Delta_T, fws_array(p_k, Delta_T))
      
      if triplet not in events_clusters.keys():
        events_clusters[triplet] = []
        events_clusters[triplet].append(p_k) 
      else:
        events_clusters[triplet].append(p_k) 
      p_k1 = tuple(p_k)
      if p_k1 not in events_triplets.keys():
        events_triplets[p_k1] = []
        events_triplets[p_k1].append(triplet)
      else:
        events_triplets[p_k1].append(triplet)

In [110]:
N = 20 # optimal clusters

In [111]:
optimal_clusters = {}
for k,v in events_clusters.items():
  if len(v)>= 20:
    optimal_clusters[k] = v

In [ ]:
optimal_clusters

In [59]:
p_new = timestamps2[0]

In [118]:
t = 2000 #current_time
grid_cut = grid[grid<t]
p_new_triplets = []
for k in range(n):
    Delta_T = np.arange(0, grid_cut[-1], 2**k)
    Delta_T = tuple(Delta_T)
    triplet = (t, Delta_T, fws_array(p_new, Delta_T))
    p_new_triplets.append(triplet)

In [ ]:
for triplet in p_new_triplets:
  if triplet in optimal_clusters.keys():
    print('found cluster ', triplet)
  if triplet not in optimal_clusters.keys():
    ind = 0
    for i in range(len(triplet[2])):
      new_fws = triplet[2][:-i]
      new_triplet = (triplet[0], triplet[1], new_fws)
      if new_triplet in optimal_clusters.keys():
        ind = 1
        print('found cluster ', new_triplet)
      if ((new_fws =='')and(ind ==0)):
          print('no closests cluster for ', triplet)